In [1]:
import hopsworks
import pandas as pd
import joblib
import os
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2


In [2]:
fv_version=2
model_version=11
td_version=1

In [3]:
key=""
with open("api-key.txt", "r") as f:
    key = f.read().rstrip()
os.environ['HOPSWORKS_PROJECT']="deloitte"
os.environ['HOPSWORKS_HOST']="6a525ee0-91d8-11ed-9cc8-9fe82dc2b6fd.cloud.hopsworks.ai"
os.environ['HOPSWORKS_API_KEY']=key

In [4]:
import hopsworks

proj = hopsworks.login()
fs = proj.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://6a525ee0-91d8-11ed-9cc8-9fe82dc2b6fd.cloud.hopsworks.ai:443/p/119
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
fv = fs.get_feature_view(name="loans_applicants", version=fv_version)

In [6]:
fv.init_batch_scoring(training_dataset_version=td_version)

In [7]:
def retrieve_data(feature_view):
    batch_data = feature_view.get_batch_data()
    return batch_data

df = retrieve_data(fv)
df

2023-03-26 12:30:54,318 INFO: USE `deloitte_featurestore`
2023-03-26 12:30:54,827 INFO: WITH right_fg0 AS (SELECT *
FROM (SELECT `fg1`.`loan_amnt` `loan_amnt`, `fg1`.`term` `term`, `fg1`.`int_rate` `int_rate`, `fg1`.`installment` `installment`, `fg1`.`sub_grade` `sub_grade`, `fg1`.`purpose` `purpose`, `fg1`.`zip_code` `zip_code`, `fg1`.`id` `join_pk_id`, `fg1`.`issue_d` `join_evt_issue_d`, `fg0`.`earliest_cr_line_year` `earliest_cr_line_year`, `fg0`.`home_ownership` `home_ownership`, `fg0`.`annual_inc` `annual_inc`, `fg0`.`verification_status` `verification_status`, `fg0`.`dti` `dti`, `fg0`.`open_acc` `open_acc`, `fg0`.`pub_rec` `pub_rec`, `fg0`.`revol_bal` `revol_bal`, `fg0`.`revol_util` `revol_util`, `fg0`.`total_acc` `total_acc`, `fg0`.`initial_list_status` `initial_list_status`, `fg0`.`application_type` `application_type`, `fg0`.`mort_acc` `mort_acc`, `fg0`.`pub_rec_bankruptcies` `pub_rec_bankruptcies`, RANK() OVER (PARTITION BY `fg1`.`id`, `fg1`.`issue_d` ORDER BY `fg0`.`earliest_

,loan_amnt,term,int_rate,installment,sub_grade,purpose,zip_code,earliest_cr_line_year,home_ownership,annual_inc,...,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies
0,0.240506,0,0.238411,0.206493,0,0,0,0.666667,0,0.013438,...,0.002624,0.168539,0.0,0.028002,0.046845,0.154362,0,0,0.000000,0.0
1,0.189873,0,0.259836,0.164456,1,1,1,0.869565,1,0.007466,...,0.002205,0.179775,0.0,0.015500,0.059733,0.167785,1,0,0.088235,0.0
2,0.382278,0,0.201402,0.323437,14,2,1,0.913043,0,0.004945,...,0.001279,0.134831,0.0,0.009229,0.103328,0.161074,1,0,0.000000,0.0
3,0.169620,0,0.045578,0.134787,2,2,1,0.898551,0,0.006202,...,0.000260,0.056180,0.0,0.004213,0.024095,0.073826,1,0,0.000000,0.0
4,0.604430,1,0.465524,0.390880,3,2,2,0.797101,1,0.006317,...,0.003395,0.134831,0.0,0.018928,0.078225,0.275168,1,0,0.029412,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395214,0.240506,1,0.220880,0.132632,0,1,3,0.869565,0,0.004594,...,0.001563,0.056180,0.0,0.001532,0.038440,0.140940,0,0,0.000000,0.0
395215,0.518987,0,0.271523,0.450897,7,1,1,0.898551,1,0.012634,...,0.002145,0.056180,0.0,0.033310,0.107251,0.040268,1,0,0.029412,0.0
395216,0.113924,0,0.181924,0.095696,15,1,4,0.768116,0,0.006489,...,0.001756,0.157303,0.0,0.025180,0.074975,0.140940,1,0,0.000000,0.0
395217,0.518987,1,0.389170,0.320834,13,1,5,0.666667,1,0.007351,...,0.001588,0.089888,0.0,0.012091,0.060294,0.120805,1,0,0.147059,0.0


In [8]:
def get_model(project):
    mr = project.get_model_registry()
    model = mr.get_model("lending_model", version=model_version)
    model_dir = model.download()
    return joblib.load(model_dir + "/lending_model.pkl")

model = get_model(proj)

Connected. Call `.close()` to terminate connection gracefully.


In [9]:
predictions = model.predict(df)
predictions

array([1, 1, 1, ..., 1, 1, 1])

In [10]:
np.unique(predictions, return_counts=True)

(array([0, 1]), array([ 20461, 374758]))